In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
from PIL import Image
import numpy as np
from transformers import BertTokenizer
from torch.nn.utils.rnn import pad_sequence

In [63]:
# all code below is from https://www.datacamp.com/tutorial/building-a-transformer-with-py-torch:

In [64]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [65]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        # d_model: Dimensionality of the input.
        # num_heads: The number of attention heads to split the input into.
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [66]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [67]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [68]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [77]:
class Transformer(nn.Module):
    def __init__(self, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.patch_projection = nn.Linear(768, d_model)  # Add a linear projection layer
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        batch_size, src_len, _ = src.size()
        tgt_len = tgt.size(1)
        src_mask = torch.ones(batch_size, 1, 1, src_len).bool().to(src.device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        nopeak_mask = (1 - torch.triu(torch.ones(1, tgt_len, tgt_len), diagonal=1)).bool().to(tgt.device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_projected = self.patch_projection(src)  # Project the image patches to the desired dimension
        src_embedded = self.dropout(self.positional_encoding(src_projected))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [78]:
def process_image(image_path, target_size=(256, 256), patch_size=16):
    # A patch size of 16 means that each patch is 16 pixels tall and 16 pixels wide
    # Load the image
    image = Image.open(image_path)
    
    # Resize the image to the target size
    resized_image = image.resize(target_size)
    
    # Convert the image to a numpy array
    image_array = np.array(resized_image)
    # print("image_array.shape", image_array.shape)
    
    # Split the image into patches
    patches = image_array.reshape(target_size[0] // patch_size, patch_size,
                                  target_size[1] // patch_size, patch_size,
                                  image_array.shape[2]).swapaxes(1, 2).reshape(-1, patch_size, patch_size, image_array.shape[2])

    # print("patches.shape", patches.shape)
    
    # Flatten the patches
    flattened_patches = patches.reshape(patches.shape[0], -1)
    
    # Normalize the flattened patches
    normalized_patches = flattened_patches / 255.0

    # Return a tensor instead of a numpy array:
    return torch.tensor(normalized_patches, dtype=torch.float32)

# Example usage
image_path = "./flashcard.jpg"
processed_patches = process_image(image_path)

print(f"Original image shape: {np.array(Image.open(image_path)).shape}")
print(f"Processed patches shape: {processed_patches.shape}")
print(f"Processed patches: {processed_patches}")

Original image shape: (2556, 1179, 3)
Processed patches shape: torch.Size([256, 768])
Processed patches: tensor([[0.4902, 0.4980, 0.5137,  ..., 0.1020, 0.1216, 0.2471],
        [0.1020, 0.1216, 0.2471,  ..., 0.1020, 0.1137, 0.2353],
        [0.1020, 0.1137, 0.2353,  ..., 0.0980, 0.1137, 0.2275],
        ...,
        [0.0510, 0.0588, 0.1098,  ..., 0.0471, 0.0588, 0.1020],
        [0.0471, 0.0549, 0.1020,  ..., 0.0392, 0.0549, 0.0902],
        [0.0392, 0.0471, 0.0941,  ..., 0.4902, 0.4980, 0.5176]])


In [79]:
class ImageCaptionDataset(data.Dataset):
    def __init__(self, image_paths, captions, tokenizer, patch_size=16, target_size=(256, 256)):
        self.image_paths = image_paths
        self.captions = captions
        self.tokenizer = tokenizer
        self.patch_size = patch_size
        self.target_size = target_size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        caption = self.captions[index]
        image_patches = process_image(image_path, self.target_size, self.patch_size)
        # Tokenize the caption
        caption_tokens = self.tokenizer.tokenize(caption)
        # Add start and end tokens
        caption_input = ['[CLS]'] + caption_tokens
        caption_label = caption_tokens + ['[SEP]']
        # Convert tokens to ids
        caption_input_ids = self.tokenizer.convert_tokens_to_ids(caption_input)
        caption_label_ids = self.tokenizer.convert_tokens_to_ids(caption_label)
        return image_patches, torch.tensor(caption_input_ids), torch.tensor(caption_label_ids)

In [92]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example usage
# image_paths = ["./flashcard.jpg", "./not-a-flashcard.jpg"]
image_paths = ["./flashcard.jpg", "./flashcard.jpg", "./flashcard.jpg", "./not-a-flashcard.jpg", "./not-a-flashcard.jpg", "./not-a-flashcard.jpg"]
captions = ["this is a flashcard", "this is a flashcard", "this is a flashcard", "this is not a flashcard", "this is not a flashcard", "this is not a flashcard"]

def collate_fn(batch):
    image_patches = [item[0] for item in batch]
    caption_inputs = [item[1] for item in batch]
    caption_labels = [item[2] for item in batch]
    
    # Pad the caption inputs and labels
    padded_caption_inputs = pad_sequence(caption_inputs, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_caption_labels = pad_sequence(caption_labels, batch_first=True, padding_value=-100)  # -100 is ignored by CrossEntropyLoss
    
    return torch.stack(image_patches), padded_caption_inputs, padded_caption_labels

# Create the dataset and data loader
dataset = ImageCaptionDataset(image_paths, captions, tokenizer)
# data_loader = data.DataLoader(dataset, batch_size=2, shuffle=True)
# use this one when I need to pad:
data_loader = data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [93]:
# testing the stuff that's in the Dataset:
# image_patches_1 = process_image("./flashcard.jpg")
# image_patches_2 = process_image("./not-a-flashcard.jpg")
# print("image_patches_2",image_patches_2)
# ^ that's all good!

# caption = "this is a flashcard"
# caption_tokens = tokenizer.tokenize(caption)
# print("caption_tokens", caption_tokens)
# caption_ids = tokenizer.encode(caption, add_special_tokens=True)
# print("caption_ids", caption_ids)
# ^ that's all good!

In [94]:
# Define the model parameters
tgt_vocab_size = tokenizer.vocab_size
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 256  # Set max_seq_length to the number of image patches
dropout = 0.1

print("tgt_vocab_size", tgt_vocab_size)

tgt_vocab_size 30522


In [95]:
# Create the transformer model
model = Transformer(tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [96]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in data_loader:
        image_patches, caption_inputs, caption_labels = batch
        image_patches = image_patches.float()

        # print("image_patches", image_patches.shape)
        # print("caption_inputs", caption_inputs.shape)
        # print("caption_labels", caption_labels.shape)

        # Forward pass
        output = model(image_patches, caption_inputs)
        
        # Compute loss and perform backward pass
        loss = criterion(output.view(-1, tgt_vocab_size), caption_labels.view(-1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 6.0499
Epoch [2/10], Loss: 4.1341
Epoch [3/10], Loss: 2.6640
Epoch [4/10], Loss: 1.1323
Epoch [5/10], Loss: 0.6918
Epoch [6/10], Loss: 0.6636
Epoch [7/10], Loss: 0.3595
Epoch [8/10], Loss: 0.3663
Epoch [9/10], Loss: 0.2378
Epoch [10/10], Loss: 0.1614


In [97]:
torch.save(model.state_dict(), 'trained_model.pth')

In [98]:
# Create a new instance of the model
trained_model = Transformer(tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Load the trained model's state dictionary
trained_model.load_state_dict(torch.load('trained_model.pth'))

# Set the model to evaluation mode
trained_model.eval()

Transformer(
  (patch_projection): Linear(in_features=768, out_features=512, bias=True)
  (decoder_embedding): Embedding(30522, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (decoder_layers): ModuleList

In [101]:
# Load and preprocess the input image
input_image_path = './not-a-flashcard.jpg'
input_image_patches = process_image(input_image_path)

# Add batch dimension to the image patches
input_image_patches = input_image_patches.unsqueeze(0)

print("input_image_patches", input_image_patches)

input_image_patches tensor([[[0.5294, 0.5137, 0.4706,  ..., 0.5333, 0.5176, 0.4745],
         [0.5333, 0.5176, 0.4745,  ..., 0.5373, 0.5216, 0.4784],
         [0.5373, 0.5216, 0.4863,  ..., 0.5412, 0.5255, 0.4824],
         ...,
         [0.5451, 0.5412, 0.5255,  ..., 0.5294, 0.5255, 0.5059],
         [0.5255, 0.5216, 0.5020,  ..., 0.5216, 0.5098, 0.4784],
         [0.5176, 0.5020, 0.4627,  ..., 0.5294, 0.5137, 0.4706]]])


In [102]:
# Set the maximum length for the generated caption
max_caption_length = 20

# Initialize the caption with the start token
generated_caption = [tokenizer.cls_token_id]

with torch.no_grad():
    for _ in range(max_caption_length):
        # Convert the generated caption to a tensor
        caption_tensor = torch.tensor(generated_caption).unsqueeze(0)

        # Generate the next token in the caption
        output = trained_model(input_image_patches, caption_tensor)
        predicted_token = output[0, -1].argmax(dim=0)

        # Append the predicted token to the generated caption
        generated_caption.append(predicted_token.item())

        # Stop generation if the end token is predicted
        if predicted_token.item() == tokenizer.sep_token_id:
            break

# Convert the generated caption tokens to text
generated_text = tokenizer.decode(generated_caption[1:-1])

print("Generated Caption:", generated_text)

Generated Caption: this is not a flashcard
